In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import matplotlib.pyplot as plt

# Load the csv file into a pandas dataframe
df = pd.read_csv('C:/Users/nsw26/OneDrive/Desktop/***FileName***.csv') #filename

# Split the data into training and testing sets
train_size = int(len(df) * 0.8)
train_data = df[:train_size]
test_data = df[train_size:]

# Scale the input data
scaler = MinMaxScaler()
scaler.fit(train_data[['rx']])  # Fit the scaler to the training data only

# Scale the output data
output_scaler = MinMaxScaler()
output_scaler.fit(train_data[['sales']])  # Fit the output scaler to the training data only

# Hardcoded next period's Rx value
next_rx = ***NextPeriodRx*** #next period Rx est. 
scaled_next_rx = scaler.transform([[next_rx]])  # Scale the next_rx value

# Store predictions
predictions = []

# Run the Monte Carlo simulation
for i in range(100):
    # Create the neural network model in each iteration
    model = Sequential([
        Input(shape=(1,)),
        Dense(10, activation='relu'),
        Dense(1, activation='linear')
    ])

    # Compile the model
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    # Train the model
    train_scaled_input = scaler.transform(train_data[['rx']])
    train_scaled_output = output_scaler.transform(train_data[['sales']])
    model.fit(train_scaled_input, train_scaled_output, epochs=100, batch_size=1, verbose=0)
    
    # Make predictions with the scaled next_rx
    scaled_output = model.predict(scaled_next_rx)
    predicted_sales = output_scaler.inverse_transform(scaled_output)
    
    # Store the prediction
    predictions.append(predicted_sales[0][0])

# Plot the histogram
plt.hist(predictions, bins=20, color='skyblue')
plt.xlabel('Predicted Sales')
plt.ylabel('Frequency')
plt.title('Predicted Sales Distribution')
plt.grid(True)
plt.show()